In [1]:
from bs4 import BeautifulSoup 
import requests
import pandas as pd
import numpy as np
pd.set_option('display.max_colwidth', None)
import time

from tqdm.notebook import tqdm
import extruct
import pickle

In [2]:
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/16.5 Safari/605.1.15"
}

In [3]:
sitemap_url = 'https://www.nytimes.com/sitemaps/new/cooking.xml.gz'
response = requests.get(sitemap_url, headers=headers)
sitemap_xml = response.text
sitemap_soup = BeautifulSoup(sitemap_xml, features='xml')

gz_urls = []
loc_tags = sitemap_soup.find_all('loc')

for loc in loc_tags:
    gz_urls.append(loc.get_text()) 

display(gz_urls)
print(len(gz_urls))

['https://www.nytimes.com/sitemaps/new/cooking-1994-02.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2020-07.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-1992-08.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2013-07.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2023-07.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2021-06.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2022-01.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2021-03.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2021-04.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2019-09.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2017-10.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2018-09.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2018-10.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2019-02.xml.gz',
 'https://www.nytimes.com/sitemaps/new/cooking-2019-06.xml.gz',
 'https://www.nytimes.com/sitemaps/new/c

497


In [4]:
with open('../data/nytc_raw_urls.pkl', 'rb') as f:
    recipe_urls = pickle.load(f)

In [5]:
# recipe_urls = []

# for url in tqdm(gz_urls):
#     response = requests.get(url, headers=headers)
#     sitemap_soup = BeautifulSoup(response.text, features='xml')
#     loc_tags = sitemap_soup.find_all('loc')
#     for loc in loc_tags:
#         recipe_urls.append(loc.get_text()) 


In [6]:
display(recipe_urls)
len(recipe_urls)

['https://cooking.nytimes.com/recipes/1024397-crispy-potato-tacos',
 'https://cooking.nytimes.com/recipes/1024394-cashew-celery',
 'https://cooking.nytimes.com/recipes/1024372-basil-butter-pasta',
 'https://cooking.nytimes.com/recipes/1024334-turmeric-potato-salad',
 'https://cooking.nytimes.com/recipes/1024222-street-corn-pudding',
 'https://cooking.nytimes.com/recipes/1024129-gorditas-de-maiz',
 'https://cooking.nytimes.com/recipes/1024370-zucchini-tofu-udon',
 'https://cooking.nytimes.com/recipes/1024398-soy-butter-corn-ramen',
 'https://cooking.nytimes.com/recipes/1024395-salt-and-vinegar-kale-chips-with-fried-chickpeas-and-avocado',
 'https://cooking.nytimes.com/recipes/1024365-bloody-mary-tomato-salad',
 'https://cooking.nytimes.com/recipes/1024373-grilled-steak-with-sauce-rof',
 'https://cooking.nytimes.com/recipes/1024284-turkey-sandwich-with-savory-blueberry-jam',
 'https://cooking.nytimes.com/recipes/1024371-paneer-chile-dry',
 'https://cooking.nytimes.com/recipes/1024374-ove

22830

In [7]:
# with open('../data/nytc_raw_urls.pkl', 'wb') as f:
#     pickle.dump(recipe_urls, f)

Quick and dirty prune?

In [8]:
import random

random_url = random.choice(recipe_urls)
print(random_url)

https://cooking.nytimes.com/recipes/4980-salt-crusted-shrimp


In [9]:
import extruct

page = requests.get(random_url, headers=headers)
recipe_soup = BeautifulSoup(page.text, 'html.parser')

recipe_data = extruct.extract(
            page.text,
            syntaxes=['json-ld'],
            uniform=True,
        )['json-ld'][0]
display(recipe_data)

{'@context': 'http://schema.org',
 '@type': 'Recipe',
 'name': 'Salt-Crusted Shrimp',
 'description': '',
 'author': {'@type': 'Person', 'name': 'Amanda Hesser'},
 'image': 'https://static01.nyt.com/applications/cooking/fc65b3a/assets/5.jpg?1',
 'totalTime': 'PT10M',
 'recipeYield': '4 appetizers',
 'recipeCuisine': '',
 'recipeCategory': 'dinner, easy, quick, appetizer',
 'keywords': 'coarse sea salt, shrimp',
 'aggregateRating': {'@type': 'AggregateRating',
  'ratingValue': 4,
  'ratingCount': 6},
 'nutrition': {'@context': 'http://schema.org',
  '@type': 'NutritionInformation',
  'calories': 128,
  'unsaturatedFatContent': '4 grams',
  'carbohydrateContent': '16 grams',
  'cholesterolContent': None,
  'fatContent': '5 grams',
  'fiberContent': '1 gram',
  'proteinContent': '4 grams',
  'saturatedFatContent': '0 grams',
  'sodiumContent': '85 milligrams',
  'sugarContent': '0 grams',
  'transFatContent': '0 grams'},
 'recipeIngredient': ['Vegetable oil or peanut oil, for frying',
  '

In [10]:
headline = recipe_data['name'] # or 'name'
nutrition = recipe_data['nutrition']
category = recipe_data['recipeCategory']
cuisine = recipe_data['recipeCuisine']
ingredient = recipe_data['recipeIngredient']
instruction = recipe_data['recipeInstructions'][0]['text']

print(headline)
print(nutrition)
print(category)
print(cuisine)
print(ingredient)
print(instruction)

Salt-Crusted Shrimp
{'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 128, 'unsaturatedFatContent': '4 grams', 'carbohydrateContent': '16 grams', 'cholesterolContent': None, 'fatContent': '5 grams', 'fiberContent': '1 gram', 'proteinContent': '4 grams', 'saturatedFatContent': '0 grams', 'sodiumContent': '85 milligrams', 'sugarContent': '0 grams', 'transFatContent': '0 grams'}
dinner, easy, quick, appetizer

['Vegetable oil or peanut oil, for frying', '1/2 cup flour', '3 tablespoons cracker meal', '12 medium shrimp, shells left on Coarse sea salt or kosher salt', 'Freshly ground black pepper']
Fill a deep saucepan with 2 inches of oil and heat to 350 degrees. In a bowl, combine flour and cracker meal. Dredge the shrimp thoroughly in the mixture. Shake off excess and drop into the oil. Fry until shrimp is curled and crisp, about 2 minutes. Drain on paper towels and season on both sides of the shrimp. Serve with tartar sauce on the side.


Test out a few recipes first

In [11]:
from nytc import NYTC

recipes = pd.Series(recipe_urls, name='url')
features = ['recipe_name', 'nutrition', 'category', 'cuisine', 'ingredient', 'instruction', 'raw_schema']

In [12]:
feature_df = pd.DataFrame(columns=features)
for i in tqdm(range(10)):
    url = recipes.iloc[i]
    try:
        recipe = NYTC(url)
        feature_df.loc[i] = [getattr(recipe, feature)() for feature in features]
        time.sleep(np.random.uniform(0.05, 1))
    except:
        feature_df.loc[i] = [np.nan] * len(features)


  0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
feature_df.head(3)

recipe_name  \
0  Crispy Potato Tacos   
1        Cashew Celery   
2   Basil-Butter Pasta   

                                                                                                                                                                                                                                                                                                                                                                                                     nutrition  \
0  {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 656, 'unsaturatedFatContent': '14 grams', 'carbohydrateContent': '86 grams', 'cholesterolContent': None, 'fatContent': '27 grams', 'fiberContent': '12 grams', 'proteinContent': '22 grams', 'saturatedFatContent': '10 grams', 'sodiumContent': '1201 milligrams', 'sugarContent': '8 grams', 'transFatContent': '0 grams'}   
1     {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 299, 'unsaturatedFatContent': '15 grams', 'carbohydrateContent': '19 grams', 'cholesterolContent': None, 'fatContent': '21 grams', 'fiberContent': '5 grams', 'proteinContent': '12 grams', 'saturatedFatContent': '3 grams', 'sodiumContent': '507 milligrams', 'sugarContent': '4 grams', 'transFatContent': '0 grams'}   
2     {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 578, 'unsaturatedFatContent': '7 grams', 'carbohydrateContent': '85 grams', 'cholesterolContent': None, 'fatContent': '19 grams', 'fiberContent': '4 grams', 'proteinContent': '16 grams', 'saturatedFatContent': '11 grams', 'sodiumContent': '355 milligrams', 'sugarContent': '3 grams', 'transFatContent': '0 grams'}   

                                                  category  cuisine  \
0                    dinner, tacos, appetizer, main course  mexican   
1  dinner, easy, quick, weeknight, vegetables, main course            
2             dinner, easy, weeknight, pastas, main course            

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                              ingredient  \
0  [Sea salt, 1 1/2 pounds potatoes (any variety), scrubbed and cut into 1 1/2-inch pieces, 1 1/2 cups grated Cheddar, Handful of cilantro, leaves and stems finely chopped, 1 small garlic clove, finely chopped, 1 teaspoon ground cumin, 1 teaspoon paprika, 16 to 18 corn tortillas, Neutral oil, as needed, Any combination of sliced lettuce or cabbage, very finely sliced red onion or sour cream (all optional), for serving, 3 tomatoes (about 1 pound), chopped, 1/2 red onion, roughly chopped, Small handful of cilantro, leaves and stems roughly chopped, 1 fresh serrano or Fresno chile (seeded, if you prefer less spice), 1 garlic clove, chopped, 1 teaspoon ground cumin, 1 teaspoon dried oregano, 1 teaspoon granulated sugar, Sea salt, 3/4 cup vegetable stock]   
1                                                                [2 teaspoons cornstarch, 1/4 cup vegetable stock or water, 2 1/2 tablespoons Shaoxing rice wine, 4 teaspoons soy sauce or tamari, Neutral oil, 4 to 6 celery stalks (about 8 ounces), trimmed and sliced diagonally, 1 (1-inch) piece fresh ginger, peeled and finely chopped, 2 garlic cloves, finely chopped, 3 scallions, trimmed and sliced, 6 ounces dark, leafy greens (such as gai lan, choy sum, kale or spinach), halved l

In [14]:
feature_df = pd.DataFrame(columns=features)
for i in tqdm(range(len(recipe_urls))):
    url = recipes.iloc[i]
    try:
        recipe = NYTC(url)
        feature_df.loc[i] = [getattr(recipe, feature)() for feature in features]
        time.sleep(np.random.uniform(0.05, 1))
    except:
        feature_df.loc[i] = [np.nan] * len(features)

  0%|          | 0/22830 [00:00<?, ?it/s]

In [15]:
data_df = pd.concat([recipes, feature_df], axis=1)

In [16]:
data_df.to_pickle('../data/nytc_data.pkl')

Sanity Check

In [17]:
data_df.sample(10)


url  \
7886                            https://cooking.nytimes.com/recipes/1015074-savory-oatmeal-pan-bread   
13898                          https://cooking.nytimes.com/recipes/6192-david-ruggieros-beet-risotto   
18480                         https://cooking.nytimes.com/recipes/2514-fried-chicken-the-yellow-rose   
16331         https://cooking.nytimes.com/recipes/4737-lamb-stew-with-chickpeas-and-butternut-squash   
5732                                     https://cooking.nytimes.com/recipes/1018246-chocolate-burfi   
11368   https://cooking.nytimes.com/recipes/10761-mango-iced-tea-with-lemon-lime-and-honey-ice-cubes   
13720  https://cooking.nytimes.com/recipes/8330-sauteed-brussels-sprouts-with-pancetta-and-chestnuts   
18806                           https://cooking.nytimes.com/recipes/1635-southwest-vegetable-melange   
17658                  https://cooking.nytimes.com/recipes/1470-roasted-potatoes-with-figs-and-thyme   
17127                                   https://cooking.nytimes.com/recipes/3859-apricot-tarte-tatin   

                                                recipe_name  \
7886                               Savory Oatmeal Pan Bread   
13898                         David Ruggiero's Beet Risotto   
18480                       Fried Chicken (The Yellow Rose)   
16331         Lamb Stew With Chickpeas and Butternut Squash   
5732                                        Chocolate Burfi   
11368   Mango Iced Tea With Lemon, Lime And Honey Ice Cubes   
13720  Sauteed Brussels Sprouts With Pancetta And Chestnuts   
18806                           Southwest Vegetable Melange   
17658                  Roasted Potatoes With Figs and Thyme   
17127                                   Apricot Tarte Tatin   

                                                                                                                                                                                                                                                                                                                                                                                                         nutrition  \
7886          {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 140, 'unsaturatedFatContent': '6 grams', 'carbohydrateContent': '14 grams', 'cholesterolContent': None, 'fatContent': '8 grams', 'fiberContent': '2 grams', 'proteinContent': '4 grams', 'saturatedFatContent': '2 grams', 'sodiumContent': '130 milligrams', 'sugarContent': '1 gram', 'transFatContent': '0 grams'}   
13898     {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 511, 'unsaturatedFatContent': '18 grams', 'carbohydrateContent': '49 grams', 'cholesterolContent': None, 'fatContent': '30 grams', 'fiberContent': '3 grams', 'proteinContent': '10 grams', 'saturatedFatContent': '10 grams', 'sodiumContent': '651 milligrams', 'sugarContent': '5 grams', 'transFatContent': '1 gram'}   
18480  {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 1409, 'unsaturatedFatContent': '86 grams', 'carbohydrateContent': '57 grams', 'cholesterolContent': None, 'fatContent': '107 grams', 'fiberContent': '2 grams', 'proteinContent': '52 grams', 'saturatedFatContent': '15 grams', 'sodiumContent': '1152 milligrams', 'sugarContent': '3 grams', 'transFatContent': '1 gram'}   
16331   {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 543, 'unsaturatedFatContent': '12 grams', 'carbohydrateContent': '58 grams', 'cholesterolContent': None, 'fatContent': '18 grams', 'fiberContent': '11 grams', 'proteinContent': '41 grams', 'saturatedFatContent': '4 grams', 'sodiumContent': '975 milligrams', 'sugarContent': '11 grams', 'transFatContent': '0 grams'}   
5732             {'@context': 'http://schema.org', '@type': 'NutritionInformation', 'calories': 285, 'unsaturatedFatContent': '5 grams', 'carbohydrateContent': '35 grams', 'cholesterolContent': None, 'fatContent': '15 grams', 'f